#   T,N,Array{T,N} ????
Why do I keep seeing this pattern in type parameters ??? <br>
Inspired by https://www.youtube.com/watch?v=fl0g9tHeghA

## 1. Reminders about type parameters

In [65]:
struct Foo{S,T,U}
    x::S
    y::T
    z::U
end

In [66]:
a = Foo(2,2.0,"two")

Foo{Int64,Float64,String}(2, 2.0, "two")

In [67]:
typeof(a)

Foo{Int64,Float64,String}

In [68]:
dump(a)

Foo{Int64,Float64,String}
  x: Int64 2
  y: Float64 2.0
  z: String "two"


## 2. Putting information about the type of the contents of foo allows us to dispatch on the contents:

In [69]:
# This doesn't work because we need the "where" syntax
import Base.*
*(a::Foo{S,T,U},b::Foo{S,T,U}) = foo(a.x+b.x, a.y+b.y, a.z+b.z)

LoadError: [91mUndefVarError: S not defined[39m

In [70]:
*(a::Foo{S,T,U},b::Foo{S,T,U}) where {S,T,U} = Foo(a.x * b.x, a.y * b.y, a.z * b.z)

* (generic function with 184 methods)

In [71]:
# This includes 
# *(a::Foo{Int64,Float64,String},b::Foo{Int64,Float64,String}) = Foo(a.x*b.x, a.y*b.y, a.z*b.z)

In [72]:
dump(a*a)

Foo{Int64,Float64,String}
  x: Int64 4
  y: Float64 4.0
  z: String "twotwo"


In [73]:
b = Foo( [1 1;1 1], [1 0;0 1], 100)

Foo{Array{Int64,2},Array{Int64,2},Int64}([1 1; 1 1], [1 0; 0 1], 100)

In [74]:
b*b

Foo{Array{Int64,2},Array{Int64,2},Int64}([2 2; 2 2], [1 0; 0 1], 10000)

## 3. Putting information about how an object is used is equally valuable 

In [75]:
# Build our own pretend matrix of two's without storing anything but size
struct twos{T,N} <: AbstractArray{T,N}
   size :: NTuple{N,Int}
end

In [76]:
Base.getindex(A::twos{T,N},i::Int...) where {T,N} = 2*one(T)   
Base.size(A::twos) = A.size

twos(::Type{T},i::Vararg{Int,N}) where {T,N} = twos{T,N}(i)
twos(i::Vararg{Int,N}) where {N} = twos{Int,N}(i)

twos

In [77]:
# primitive constructor
twos{Int,2}((3,4))

3×4 twos{Int64,2}:
 2  2  2  2
 2  2  2  2
 2  2  2  2

In [78]:
# convenience constructor
twos(Float64,3,4)

3×4 twos{Float64,2}:
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0

In [79]:
# more convenient
twos(3,4)

3×4 twos{Int64,2}:
 2  2  2  2
 2  2  2  2
 2  2  2  2

## 4. Reshape as an example

In [52]:
[1:25;]

25-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21
 22
 23
 24
 25

In [53]:
1:25

1:25

In [83]:
(1:25).start, (1:25).stop

(1, 25)

In [84]:
dump(1:25)

UnitRange{Int64}
  start: Int64 1
  stop: Int64 25


In [85]:
reshape(1:25,5,5)

5×5 Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}}:
 1   6  11  16  21
 2   7  12  17  22
 3   8  13  18  23
 4   9  14  19  24
 5  10  15  20  25

In [86]:
A = reshape(1:25,5,5)
A.parent, A.dims

(1:25, (5, 5))

In [87]:
dump(A)

Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}}
  parent: UnitRange{Int64}
    start: Int64 1
    stop: Int64 25
  dims: Tuple{Int64,Int64}
    1: Int64 5
    2: Int64 5
  mi: Tuple{} ()


In [ ]:
arrays are natural for lazy evaluation

Tim Holy is a master of using the type system to keep track of the contortions you do on the data without doing it on the data --- this is lazy evaluation.

In [ ]:
A=[1 2 3 4]
reshape(A,2,2)

In [ ]:
A = 'a':'y'
reshape(A,5,5)

In [ ]:
A=1:25
R= reshape(A,(5,5))

In [ ]:
dump(R)

In [ ]:
R + R

In [ ]:
reshape(2:2:50,(5,5))

In [ ]:
Array(reshape(2:2:50,(5,5)))

In [ ]:
Base.ReshapedArray <: AbstractArray

In [ ]:
UnitRange <: AbstractArray

In [ ]:
StepRange <: AbstractArray

In [ ]:
supertype(StepRange)

In [ ]:
supertype(UnitRange)

In [ ]:
reshape(1:25,(5,5))  +   reshape(2:2:50,(5,5))

In [ ]:
Base.summarysize(1:25), Base.summarysize([1:25;])

In [ ]:
# What is a Holy type?
# It is a type that has the pattern 
# foo(T,N,Array{T,N},other_stuff)

In [42]:
A = reshape([1:25;],5,5)

5×5 Array{Int64,2}:
 1   6  11  16  21
 2   7  12  17  22
 3   8  13  18  23
 4   9  14  19  24
 5  10  15  20  25

In [30]:
B = view(A,1:3,1:4)

3×4 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 1  6  11  16
 2  7  12  17
 3  8  13  18

In [ ]:
# See the pattern T,N,Array{T,N}?  T=Int64 and N=2 in Int64,2,Array{Int64,2}

In [44]:
using MappedArrays

In [ ]:
Normed{UInt8,8}

In [45]:
M = mappedarray((x->x^2,√),A)

5×5 MappedArrays.MappedArray{Int64,2,Array{Int64,2},##5#6,Base.#sqrt}:
  1   36  121  256  441
  4   49  144  289  484
  9   64  169  324  529
 16   81  196  361  576
 25  100  225  400  625

In [46]:
M = mappedarray(√,A)

5×5 MappedArrays.ReadonlyMappedArray{Float64,2,Array{Int64,2},Base.#sqrt}:
 1.0      2.44949  3.31662  4.0      4.58258
 1.41421  2.64575  3.4641   4.12311  4.69042
 1.73205  2.82843  3.60555  4.24264  4.79583
 2.0      3.0      3.74166  4.3589   4.89898
 2.23607  3.16228  3.87298  4.47214  5.0    

In [ ]:
typeof(M)

In [ ]:
supertype(typeof(M))

In [ ]:
supertype(supertype(typeof(M)))

In [ ]:
A=reshape([1:25;],(5,5))

In [ ]:
M = mappedarray((sqrt,x->x^2),A)

In [ ]:
M[1,1]= 9.0


In [ ]:
M

In [ ]:
A

In [ ]:
M =  vec(mappedarray(√,A))

In [ ]:
struct Something{S,T}
     X :: S
     Y :: T
end

In [ ]:
Something(4.4,5)

In [ ]:
struct Thing{T,S}
     X :: S
     Y :: T
end

In [ ]:
Thing(4.4,5)

In [ ]:
struct NewThing{T}
     X :: Float64
     Y :: T
end

In [ ]:
NewThing(5.5,45)

In [ ]:
A = rand(2,2)

In [ ]:
struct Thing3{T}
     X::Array{T,2}
end

In [ ]:
Thing3(A)

In [ ]:
struct Thing6{T}
     X::Array{Array{T,2},1}
end

In [ ]:
typeof(Thing6( [rand(5,5) for i=1:3]))

In [ ]:
Thing6( [rand(5,5) for i=1:3])

In [ ]:
##3